# BG/NBD & Gamma Gamma Modelleri ile CLTV Prediction

## Kütüphanelerin Yüklenmesi

In [61]:
import datetime as dt 
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

## Verimizi Okuyalım

In [62]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1000,12680.0000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.9500,12680.0000,France


## VeriSetinde ki Aykırı değerleri baskılamak için fonksiyon tanımlıyoruz 

In [63]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


## Veri Önİşleme

In [64]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1000,12680.0000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.9500,12680.0000,France


In [65]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [66]:
df.dropna(inplace=True)

In [67]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

### Verisetinin istatiklerine baktığımız zaman aykırı değer veya anormali durumların olup olmadığını görebiliriz

In [68]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,406830.0000,12.0613,248.6931,-80995.0000,2.0000,5.0000,12.0000,80995.0000
Price,406830.0000,3.4605,69.3151,0.0000,1.2500,1.9500,3.7500,38970.0000
Customer ID,406830.0000,15287.6842,1713.6031,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000


In [69]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1000,12680.0000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.9500,12680.0000,France


In [70]:
df = df[~df['Invoice'].str.contains('C', na=False)]

In [71]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1000,12680.0000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.1500,12680.0000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.9500,12680.0000,France


In [72]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,13.0218,180.4200,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397925.0000,3.1162,22.0968,0.0000,1.2500,1.9500,3.7500,8142.7500
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [73]:
df = df[df['Quantity']>0]
df = df[df['Price']>0]

In [74]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,12.9882,179.3316,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397885.0000,3.1165,22.0979,0.0010,1.2500,1.9500,3.7500,8142.7500
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [75]:
replace_with_thresholds(df, 'Quantity')
replace_with_thresholds(df, 'Price')

In [76]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,11.8308,25.5231,1.0000,2.0000,6.0000,12.0000,298.5000
Price,397885.0000,2.8935,3.2272,0.0010,1.2500,1.9500,3.7500,37.0600
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [77]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0000,2011-12-09 12:50:00,2.1000,12680.0000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0000,2011-12-09 12:50:00,4.1500,12680.0000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0000,2011-12-09 12:50:00,4.1500,12680.0000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3.0000,2011-12-09 12:50:00,4.9500,12680.0000,France


In [78]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [79]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0000,2011-12-09 12:50:00,2.1000,12680.0000,France,12.6000
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0000,2011-12-09 12:50:00,4.1500,12680.0000,France,16.6000
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0000,2011-12-09 12:50:00,4.1500,12680.0000,France,16.6000
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3.0000,2011-12-09 12:50:00,4.9500,12680.0000,France,14.8500


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397885 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      397885 non-null  object        
 1   StockCode    397885 non-null  object        
 2   Description  397885 non-null  object        
 3   Quantity     397885 non-null  float64       
 4   InvoiceDate  397885 non-null  datetime64[ns]
 5   Price        397885 non-null  float64       
 6   Customer ID  397885 non-null  float64       
 7   Country      397885 non-null  object        
 8   TotalPrice   397885 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 30.4+ MB


In [81]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [82]:
analysis_date = dt.datetime(2011,12,11)
analysis_date

datetime.datetime(2011, 12, 11, 0, 0)

## Lifetime veri yapısının hazırlanması

In [83]:
cltv_c = df.groupby('Customer ID').agg({'InvoiceDate' : [lambda date: (date.max() - date.min()).days,
                                                        lambda x: (analysis_date - x.min()).days],
                                        'Invoice' : lambda num: num.nunique(),
                                        'TotalPrice' : lambda x: x.sum()})

In [84]:
cltv_c

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000            0        326        1   310.4400
12347.0000          365        368        7  4310.0000
12348.0000          282        359        4  1770.7800
12349.0000            0         19        1  1491.7200
12350.0000            0        311        1   331.4600
...                 ...        ...      ...        ...
18280.0000            0        278        1   180.6000
18281.0000            0        181        1    80.8200
18282.0000          118        127        2   178.0500
18283.0000          333        338       16  2094.8800
18287.0000          158        202        3  1837.2800

[4338 rows x 4 columns]

In [85]:
cltv_c.columns = cltv_c.columns.droplevel(0)

In [86]:
cltv_c

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.0000,0,326,1,310.4400
12347.0000,365,368,7,4310.0000
12348.0000,282,359,4,1770.7800
12349.0000,0,19,1,1491.7200
12350.0000,0,311,1,331.4600
...,...,...,...,...
18280.0000,0,278,1,180.6000
18281.0000,0,181,1,80.8200
18282.0000,118,127,2,178.0500


In [87]:
cltv_c.columns = ['recency','T','frequency','monetary']
cltv_c

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,310.4400
12347.0000,365,368,7,4310.0000
12348.0000,282,359,4,1770.7800
12349.0000,0,19,1,1491.7200
12350.0000,0,311,1,331.4600
...,...,...,...,...
18280.0000,0,278,1,180.6000
18281.0000,0,181,1,80.8200
18282.0000,118,127,2,178.0500


In [88]:
cltv_c['monetary'] = cltv_c['monetary'] / cltv_c['frequency']
cltv_c

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,310.4400
12347.0000,365,368,7,615.7143
12348.0000,282,359,4,442.6950
12349.0000,0,19,1,1491.7200
12350.0000,0,311,1,331.4600
...,...,...,...,...
18280.0000,0,278,1,180.6000
18281.0000,0,181,1,80.8200
18282.0000,118,127,2,89.0250


In [89]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4338.0000,130.4486,132.0396,0.0000,0.0000,92.5000,251.7500,373.0000
T,4338.0000,223.8310,117.8546,1.0000,113.0000,249.0000,327.0000,374.0000
frequency,4338.0000,4.2720,7.6980,1.0000,1.0000,2.0000,5.0000,209.0000
monetary,4338.0000,364.1185,367.2582,3.4500,176.8512,288.2255,422.0294,6207.6700


In [90]:
cltv_c = cltv_c[(cltv_c['frequency'] > 1)]

In [91]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2845.0000,198.9054,113.8670,0.0000,98.0000,207.0000,299.0000,373.0000
T,2845.0000,258.7842,102.9909,3.0000,192.0000,283.0000,359.0000,374.0000
frequency,2845.0000,5.9891,9.0442,2.0000,2.0000,4.0000,6.0000,209.0000
monetary,2845.0000,372.1736,349.3012,3.4500,191.0733,300.4873,436.8700,5724.3026


In [92]:
cltv_c['recency'] = cltv_c['recency'] / 7
cltv_c['T'] = cltv_c['T'] / 7

In [93]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2845.0000,28.4151,16.2667,0.0000,14.0000,29.5714,42.7143,53.2857
T,2845.0000,36.9692,14.7130,0.4286,27.4286,40.4286,51.2857,53.4286
frequency,2845.0000,5.9891,9.0442,2.0000,2.0000,4.0000,6.0000,209.0000
monetary,2845.0000,372.1736,349.3012,3.4500,191.0733,300.4873,436.8700,5724.3026


In [94]:
cltv_c

,recency,T,frequency,monetary
Customer ID,,,,
12347.0000,52.1429,52.5714,7,615.7143
12348.0000,40.2857,51.2857,4,442.6950
12352.0000,37.1429,42.4286,8,219.5425
12356.0000,43.1429,46.5714,3,937.1433
12358.0000,21.2857,21.5714,2,575.2100
...,...,...,...,...
18272.0000,34.8571,35.2857,6,513.0967
18273.0000,36.4286,36.8571,3,68.0000
18282.0000,16.8571,18.1429,2,89.0250


## BG/NBD Modeli

In [95]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [96]:
bgf.fit(cltv_c['frequency'],
        cltv_c['recency'],
        cltv_c['T'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.41, b: 2.49, r: 2.18>

In [97]:
cltv_c['exp_sales_6_month'] = bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                                                    cltv_c['frequency'],
                                                                                    cltv_c['recency'],
                                                                                    cltv_c['T'])

In [98]:
cltv_c

,recency,T,frequency,monetary,exp_sales_6_month
Customer ID,,,,,
12347.0000,52.1429,52.5714,7,615.7143,3.3239
12348.0000,40.2857,51.2857,4,442.6950,2.1618
12352.0000,37.1429,42.4286,8,219.5425,4.2775
12356.0000,43.1429,46.5714,3,937.1433,2.0219
12358.0000,21.2857,21.5714,2,575.2100,2.8279
...,...,...,...,...,...
18272.0000,34.8571,35.2857,6,513.0967,4.0220
18273.0000,36.4286,36.8571,3,68.0000,2.4390
18282.0000,16.8571,18.1429,2,89.0250,3.1274


In [99]:
cltv_c.sort_values(by='exp_sales_6_month', ascending=False).head(10)

,recency,T,frequency,monetary,exp_sales_6_month
Customer ID,,,,,
12748.0000,53.1429,53.4286,209,154.9302,76.5418
14911.0000,53.1429,53.4286,201,691.7101,73.6417
17841.0000,53.0000,53.4286,124,330.1344,45.6983
13089.0000,52.2857,52.8571,97,606.3625,36.2063
14606.0000,53.1429,53.4286,93,130.1390,34.4798
15311.0000,53.2857,53.4286,91,667.7791,33.7665
12971.0000,52.5714,53.2857,86,127.4859,31.9589
14646.0000,50.4286,50.7143,73,3646.0757,28.3924
13408.0000,53.0000,53.4286,62,453.5006,23.2269


In [100]:
cltv_c['exp_sales_1_month'] = bgf.predict(4*1,
                                          cltv_c['frequency'],
                                          cltv_c['recency'],
                                          cltv_c['T'])

In [101]:
cltv_c['exp_sales_12_month'] = bgf.predict(4*12,
                                          cltv_c['frequency'],
                                          cltv_c['recency'],
                                          cltv_c['T'])

In [102]:
cltv_c.sort_values(by='exp_sales_1_month', ascending=False).head(10)

,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month
Customer ID,,,,,,,
12748.0000,53.1429,53.4286,209,154.9302,76.5418,12.9633,150.6340
14911.0000,53.1429,53.4286,201,691.7101,73.6417,12.4722,144.9265
17841.0000,53.0000,53.4286,124,330.1344,45.6983,7.7398,89.9322
13089.0000,52.2857,52.8571,97,606.3625,36.2063,6.1330,71.2436
14606.0000,53.1429,53.4286,93,130.1390,34.4798,5.8399,67.8534
15311.0000,53.2857,53.4286,91,667.7791,33.7665,5.7191,66.4497
12971.0000,52.5714,53.2857,86,127.4859,31.9589,5.4131,62.8905
14646.0000,50.4286,50.7143,73,3646.0757,28.3924,4.8119,55.8439
13408.0000,53.0000,53.4286,62,453.5006,23.2269,3.9341,45.7070


In [103]:
cltv_c.sort_values(by='exp_sales_12_month', ascending=False).head(10)

,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month
Customer ID,,,,,,,
12748.0000,53.1429,53.4286,209,154.9302,76.5418,12.9633,150.6340
14911.0000,53.1429,53.4286,201,691.7101,73.6417,12.4722,144.9265
17841.0000,53.0000,53.4286,124,330.1344,45.6983,7.7398,89.9322
13089.0000,52.2857,52.8571,97,606.3625,36.2063,6.1330,71.2436
14606.0000,53.1429,53.4286,93,130.1390,34.4798,5.8399,67.8534
15311.0000,53.2857,53.4286,91,667.7791,33.7665,5.7191,66.4497
12971.0000,52.5714,53.2857,86,127.4859,31.9589,5.4131,62.8905
14646.0000,50.4286,50.7143,73,3646.0757,28.3924,4.8119,55.8439
13408.0000,53.0000,53.4286,62,453.5006,23.2269,3.9341,45.7070


## Gamma Gamma Modeli

In [104]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [105]:
ggf.fit(cltv_c['frequency'], 
        cltv_c['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.79, q: 0.34, v: 3.73>

In [106]:
ggf.conditional_expected_average_profit(cltv_c['frequency'], 
                                        cltv_c['monetary']).sort_values(ascending=False).head(10)

Customer ID
12415.0000   5772.1782
12590.0000   5029.4196
12435.0000   4288.9440
12409.0000   3918.8128
14088.0000   3917.1297
18102.0000   3870.9969
12753.0000   3678.5783
14646.0000   3654.8148
15749.0000   3216.0523
14096.0000   3196.4361
dtype: float64

In [107]:
cltv_c['expected_average_profit'] = ggf.conditional_expected_average_profit(cltv_c['frequency'], 
                                                                            cltv_c['monetary'])

In [108]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_c['frequency'],
                                   cltv_c['recency'],
                                   cltv_c['T'],
                                   cltv_c['monetary'],
                                   time=1,  #  aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

In [109]:
cltv.sort_values(ascending=False).head(10)

Customer ID
14646.0000   18908.3674
18102.0000   16080.1149
14096.0000    9946.8961
12415.0000    9395.0400
14911.0000    9283.9046
17450.0000    9213.6136
14156.0000    7944.0199
17511.0000    6439.1262
16684.0000    4559.2868
16000.0000    4430.9443
Name: clv, dtype: float64

In [110]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_c['frequency'],
                                   cltv_c['recency'],
                                   cltv_c['T'],
                                   cltv_c['monetary'],
                                   time=12,  #  aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

In [111]:
cltv.sort_values(ascending=False).head(10)

Customer ID
14646.0000   207547.0487
18102.0000   176638.6550
14096.0000   105747.5034
12415.0000   102959.5167
14911.0000   102037.3098
17450.0000   101203.4616
14156.0000    87283.6445
17511.0000    70743.7497
16684.0000    50042.1512
13694.0000    48317.6405
Name: clv, dtype: float64

In [112]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_c['frequency'],
                                   cltv_c['recency'],
                                   cltv_c['T'],
                                   cltv_c['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

In [113]:
cltv.sort_values(ascending=False).head(10)

Customer ID
14646.0000   108728.3152
18102.0000    92504.4138
14096.0000    56127.5032
12415.0000    53976.1227
14911.0000    53423.4139
17450.0000    53001.2495
14156.0000    45705.2997
17511.0000    37045.5459
16684.0000    26216.3827
13694.0000    25300.0288
Name: clv, dtype: float64

In [114]:
cltv = cltv.reset_index()
cltv

,Customer ID,clv
0,12347.0000,2200.7340
1,12348.0000,1050.3748
2,12352.0000,1007.7137
3,12356.0000,2109.6204
4,12358.0000,1870.5449
...,...,...
2840,18272.0000,2228.5118
2841,18273.0000,187.7430
2842,18282.0000,325.7606
2843,18283.0000,985.5124


In [115]:
cltv_final = cltv_c.merge(cltv, on='Customer ID', how='left')
cltv_final

,Customer ID,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month,expected_average_profit,clv
0,12347.0000,52.1429,52.5714,7,615.7143,3.3239,0.5635,6.5358,631.9123,2200.7340
1,12348.0000,40.2857,51.2857,4,442.6950,2.1618,0.3668,4.2483,463.7460,1050.3748
2,12352.0000,37.1429,42.4286,8,219.5425,4.2775,0.7271,8.3928,224.8868,1007.7137
3,12356.0000,43.1429,46.5714,3,937.1433,2.0219,0.3435,3.9684,995.9989,2109.6204
4,12358.0000,21.2857,21.5714,2,575.2100,2.8279,0.4862,5.5024,631.9022,1870.5449
...,...,...,...,...,...,...,...,...,...,...
2840,18272.0000,34.8571,35.2857,6,513.0967,4.0220,0.6856,7.8746,529.0185,2228.5118
2841,18273.0000,36.4286,36.8571,3,68.0000,2.4390,0.4157,4.7751,73.4942,187.7430
2842,18282.0000,16.8571,18.1429,2,89.0250,3.1274,0.5392,6.0743,99.5249,325.7606
2843,18283.0000,47.5714,48.2857,16,130.9300,7.0936,1.2034,13.9412,132.6012,985.5124


In [116]:
cltv_final.sort_values(by='clv', ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month,expected_average_profit,clv
1122,14646.0000,50.4286,50.7143,73,3646.0757,28.3924,4.8119,55.8439,3654.8148,108728.3152
2761,18102.0000,52.2857,52.5714,60,3859.7391,22.8061,3.8636,44.8717,3870.9969,92504.4138
843,14096.0000,13.8571,14.5714,17,3163.5882,16.7786,2.8955,32.5718,3196.4361,56127.5032
36,12415.0000,44.7143,48.2857,21,5724.3026,8.9251,1.5139,17.5421,5772.1782,53976.1227
1257,14911.0000,53.1429,53.4286,201,691.7101,73.6417,12.4722,144.9265,692.3264,53423.4139
2458,17450.0000,51.2857,52.5714,46,2863.2749,17.5988,2.9815,34.6250,2874.1987,53001.2495
874,14156.0000,51.5714,53.1429,55,2104.0267,20.6651,3.5005,40.6631,2110.7542,45705.2997
2487,17511.0000,52.8571,53.4286,31,2933.9431,11.9822,2.0298,23.5770,2950.5801,37045.5459
2075,16684.0000,50.4286,51.2857,28,2209.9691,11.2509,1.9068,22.1284,2223.8850,26216.3827
650,13694.0000,52.7143,53.4286,50,1275.7005,18.8601,3.1946,37.1130,1280.2183,25300.0288


In [117]:
cltv_final['segment'] = pd.qcut(cltv_final['clv'], 4, labels=['D','C','B','A'])

In [118]:
cltv_final

,Customer ID,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month,expected_average_profit,clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,3.3239,0.5635,6.5358,631.9123,2200.7340,A
1,12348.0000,40.2857,51.2857,4,442.6950,2.1618,0.3668,4.2483,463.7460,1050.3748,B
2,12352.0000,37.1429,42.4286,8,219.5425,4.2775,0.7271,8.3928,224.8868,1007.7137,B
3,12356.0000,43.1429,46.5714,3,937.1433,2.0219,0.3435,3.9684,995.9989,2109.6204,A
4,12358.0000,21.2857,21.5714,2,575.2100,2.8279,0.4862,5.5024,631.9022,1870.5449,A
...,...,...,...,...,...,...,...,...,...,...,...
2840,18272.0000,34.8571,35.2857,6,513.0967,4.0220,0.6856,7.8746,529.0185,2228.5118,A
2841,18273.0000,36.4286,36.8571,3,68.0000,2.4390,0.4157,4.7751,73.4942,187.7430,D
2842,18282.0000,16.8571,18.1429,2,89.0250,3.1274,0.5392,6.0743,99.5249,325.7606,D
2843,18283.0000,47.5714,48.2857,16,130.9300,7.0936,1.2034,13.9412,132.6012,985.5124,C


In [119]:
cltv_final.sort_values(by="clv", ascending=False).head(50)


,Customer ID,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month,expected_average_profit,clv,segment
1122,14646.0000,50.4286,50.7143,73,3646.0757,28.3924,4.8119,55.8439,3654.8148,108728.3152,A
2761,18102.0000,52.2857,52.5714,60,3859.7391,22.8061,3.8636,44.8717,3870.9969,92504.4138,A
843,14096.0000,13.8571,14.5714,17,3163.5882,16.7786,2.8955,32.5718,3196.4361,56127.5032,A
36,12415.0000,44.7143,48.2857,21,5724.3026,8.9251,1.5139,17.5421,5772.1782,53976.1227,A
1257,14911.0000,53.1429,53.4286,201,691.7101,73.6417,12.4722,144.9265,692.3264,53423.4139,A
2458,17450.0000,51.2857,52.5714,46,2863.2749,17.5988,2.9815,34.6250,2874.1987,53001.2495,A
874,14156.0000,51.5714,53.1429,55,2104.0267,20.6651,3.5005,40.6631,2110.7542,45705.2997,A
2487,17511.0000,52.8571,53.4286,31,2933.9431,11.9822,2.0298,23.5770,2950.5801,37045.5459,A
2075,16684.0000,50.4286,51.2857,28,2209.9691,11.2509,1.9068,22.1284,2223.8850,26216.3827,A
650,13694.0000,52.7143,53.4286,50,1275.7005,18.8601,3.1946,37.1130,1280.2183,25300.0288,A


In [120]:
cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"})

Customer ID                          recency                        T                    frequency               monetary                      exp_sales_6_month                  exp_sales_1_month                 exp_sales_12_month                   expected_average_profit                        clv                       
              count           sum       mean   count        sum    mean count        sum    mean     count   sum    mean    count         sum     mean             count       sum   mean             count      sum   mean              count       sum    mean                   count         sum     mean count          sum      mean
segment                                                                                                                                                                                                                                                                                                                                   
D               712 11074077.0000 15553.4789     712 15706.1429 22.0592   712 28775.0000 40.4143       712  2181  3.0632      712 130766.0506 183.6602               712 1181.9939 1.6601               712 201.8279 0.2835                712 2312.0574  3.2473                     712 141775.9623 199.1235   712  198346.4783  278.5765
C               711 10894987.0000 15323.4698     711 21746.1429 30.5853   711 27059.7143 38.0587       711  2910  4.0928      711 193065.6656 271.5410               711 2001.8516 2.8155               711 341.9792 0.4810                711 3914.6834  5.5059                     711 206071.1878 289.8329   711  526160.2307  740.0285
B               711 10910905.0000 15345.8579     711 20974.1429 29.4995   711 24746.4286 34.8051       711  3856  5.4233      711 265861.9647 373.9268               711 2678.7137 3.7675               711 458.7084 0.6452                711 5230.6601  7.3568                     711 280456.8023 394.4540   711  949845.8856 1335.9295
A               711 10626242.0000 14945.4880     711 22414.4286 31.5252   711 24596.1429 34.5937       711  8092 11.3812      711 469140.1053 659.8314               711 4530.3232 6.3718               711 774.3780 1.0891                711 8858.8012 12.4596                     711 487609.1259 685.8075   711 3069625.1306 4317.3349

In [121]:
cltv_final.to_csv('online_retail_cltv_prediction.csv')

In [124]:
cltv_final2 = cltv_final[(cltv_final['segment'] == 'A') | (cltv_final['segment'] == 'B')]
cltv_final2

,Customer ID,recency,T,frequency,monetary,exp_sales_6_month,exp_sales_1_month,exp_sales_12_month,expected_average_profit,clv,segment
0,12347.0000,52.1429,52.5714,7,615.7143,3.3239,0.5635,6.5358,631.9123,2200.7340,A
1,12348.0000,40.2857,51.2857,4,442.6950,2.1618,0.3668,4.2483,463.7460,1050.3748,B
2,12352.0000,37.1429,42.4286,8,219.5425,4.2775,0.7271,8.3928,224.8868,1007.7137,B
3,12356.0000,43.1429,46.5714,3,937.1433,2.0219,0.3435,3.9684,995.9989,2109.6204,A
4,12358.0000,21.2857,21.5714,2,575.2100,2.8279,0.4862,5.5024,631.9022,1870.5449,A
...,...,...,...,...,...,...,...,...,...,...,...
2834,18259.0000,48.7143,52.4286,3,779.5333,1.8402,0.3122,3.6164,828.7124,1597.7865,B
2837,18263.0000,32.2857,36.1429,3,404.3867,2.4432,0.4166,4.7822,430.5335,1101.6727,B
2838,18265.0000,12.1429,22.7143,2,400.7550,2.4415,0.4194,4.7531,440.8723,1126.7825,B
2840,18272.0000,34.8571,35.2857,6,513.0967,4.0220,0.6856,7.8746,529.0185,2228.5118,A


In [125]:
cltv_final2.to_csv('online_retail_segment_A_B.csv')